In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lstm-bax/df_bax_cleaned_to_view_outliers.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import math
from sklearn.metrics import mean_squared_error

2025-08-08 05:42:58.565815: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754631778.742756      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754631778.791873      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# --- Step 1: Load and Prepare Data ---

# Load the dataset from the CSV file
df = pd.read_csv('/kaggle/input/lstm-bax/df_bax_cleaned_to_view_outliers.csv')

# Convert the 'Date' column to datetime objects and set it as the index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Create a new dataframe with only the 'Price' column
data = df.filter(['Price'])

# Convert the dataframe to a numpy array
dataset = data.values

# Determine the training data length (e.g., 80% of the data)
training_data_len = math.ceil(len(dataset) * .8)

In [5]:
# --- Step 2: Scale the Data ---

# Scale the data to be values between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

In [6]:
# --- Step 3: Create Training Dataset and Sequences ---

# Create the scaled training data set
train_data = scaled_data[0:training_data_len, :]

# Split the data into x_train and y_train data sets
x_train = []
y_train = []
time_step = 60 # Number of past days to use for prediction

for i in range(time_step, len(train_data)):
    x_train.append(train_data[i-time_step:i, 0])
    y_train.append(train_data[i, 0])

# Convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the data to be 3D [samples, time steps, features] for the LSTM model
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [7]:
# --- Step 4: Build and Train the LSTM Model ---

# Build the LSTM model architecture
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
print("Training the LSTM model...")
model.fit(x_train, y_train, batch_size=32, epochs=20)
print("Model training complete.")

I0000 00:00:1754631893.786844      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1754631893.787563      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training the LSTM model...
Epoch 1/20


I0000 00:00:1754631898.512371     105 cuda_dnn.cc:529] Loaded cuDNN version 90300


90/90 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0085
Epoch 2/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3.6942e-04
Epoch 3/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3.6894e-04
Epoch 4/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3.5986e-04
Epoch 5/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.3509e-04
Epoch 6/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.4260e-04
Epoch 7/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.2697e-04
Epoch 8/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.8110e-04
Epoch 9/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.6624e-04
Epoch 10/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.7923e-04
Epoch 11/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.0491e-04
Epoch 12/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.6452e-04
Epoch 13/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.5145e-04
Epoch 14/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.4842e-04
Epoch 15/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 8m

In [8]:
# --- Step 5: Create Test Dataset and Make Predictions ---

# Create the testing data set
test_data = scaled_data[training_data_len - time_step:, :]

# Create the x_test and y_test data sets
x_test = []
y_test = dataset[training_data_len:, :] # Actual values for comparison

for i in range(time_step, len(test_data)):
    x_test.append(test_data[i-time_step:i, 0])

# Convert the data to a numpy array and reshape it
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Get the model's predicted price values
predictions = model.predict(x_test)

# Inverse transform the predictions to get the actual price values back
predictions = scaler.inverse_transform(predictions)

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


In [9]:
# --- Step 6: Evaluate the Model and Visualize Results ---

# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'\nRoot Mean Squared Error (RMSE): {rmse}')

# Plot the data for visualization
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16,8))
plt.title('BAX Index Price Prediction using LSTM')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price BHD', fontsize=18)
plt.plot(train['Price'])
plt.plot(valid[['Price', 'Predictions']])
plt.legend(['Train', 'Actual', 'Predictions'], loc='lower right')
plt.grid(True)
plt.show()


Root Mean Squared Error (RMSE): 26.51278859858718


/tmp/ipykernel_36/1993839136.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


In [10]:
# Add validation_split to the fit method and store the training history
history = model.fit(x_train, y_train, 
                    batch_size=32, 
                    epochs=20,        # You might need more epochs to see the effect
                    validation_split=0.2) # Use 20% of training data for validation

# Plotting the training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)
plt.show()

Epoch 1/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 8.5579e-05 - val_loss: 2.3551e-04
Epoch 2/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 8.4258e-05 - val_loss: 1.8602e-04
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 6.9765e-05 - val_loss: 1.8386e-04
Epoch 4/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 6.8380e-05 - val_loss: 2.2793e-04
Epoch 5/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 6.7300e-05 - val_loss: 2.8804e-04
Epoch 6/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 6.2183e-05 - val_loss: 1.6804e-04
Epoch 7/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 6.2205e-05 - val_loss: 1.6000e-04
Epoch 8/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 5.5245e-05 - val_loss: 1.6331e-04
Epoch 9/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 6.9598e-05 - val_loss: 1.5366e-04
Epoch 10/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 6.3046e-05 - val_loss: 1.6291e-04
Epoch 11/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 6.2829e-05 - val_lo